<a href="https://colab.research.google.com/github/lorenzocesconetto/gemma-intro/blob/master/RAG_with_Gemma_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG with Gemma 2B

## Installing the libraries we will use

In [1]:
!pip install accelerate transformers sentence-transformers faiss-gpu -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

## Download files

In [2]:
!mkdir readmes
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/transformer.md -O readmes/transformer.md
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/pymatting.md -O readmes/pymatting.md
!wget https://huggingface.co/datasets/pedrogengo/readmes/raw/main/yolo.md -O readmes/yolo.md

--2024-03-28 20:14:29--  https://huggingface.co/datasets/pedrogengo/readmes/raw/main/transformer.md
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.23, 18.164.174.17, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112946 (110K) [text/plain]
Saving to: ‘readmes/transformer.md’

readmes/transformer 100%[===================>] 110.30K  --.-KB/s    in 0.07s   

2024-03-28 20:14:30 (1.52 MB/s) - ‘readmes/transformer.md’ saved [112946/112946]

--2024-03-28 20:14:30--  https://huggingface.co/datasets/pedrogengo/readmes/raw/main/pymatting.md
Resolving huggingface.co (huggingface.co)... 18.155.173.126, 18.155.173.45, 18.155.173.64, ...
Connecting to huggingface.co (huggingface.co)|18.155.173.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6794 (6.6K) [text/plain]
Saving to: ‘readmes/pymatting.md’

readmes/pymatting.m 100%[===================>]   6.63K 

## Imports

In [3]:
import os

import markdown
import faiss
import torch
from bs4 import BeautifulSoup
from huggingface_hub import login
from sentence_transformers import SentenceTransformer

# login()

from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

## Generating chunks

In [5]:
def md_to_text(md):
    html = markdown.markdown(md)
    soup = BeautifulSoup(html, features='html.parser')
    return soup.get_text()

def split_text(document_path, chunk_size, overlap):
  with open(document_path, 'r', encoding='utf-8') as file:
        content = file.read()

  content = md_to_text(content)
  chunks = []
  for i in range(0, len(content), chunk_size):
    chunk = document_path + content[max(0, i-overlap):min(len(content), i+chunk_size)] #pymatting.md content[0:100]. -> pymatting.md content[100:200] ...
    chunks.append(chunk)
  return chunks

In [6]:
readmes_dir = "readmes"
chunk_size = 700
overlap = 100

all_chunks = []
for f in os.listdir(readmes_dir):
  if f[-2:] == "md":
    all_chunks += split_text(os.path.join(readmes_dir, f), chunk_size, overlap)

In [ ]:
len(all_chunks)

130

## Creating the embeddings

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
_ = embedding_model.to(device)
embeddings = embedding_model.encode(all_chunks)
embeddings.shape

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(130, 384)

## Indexing embeddings using FAISS

In [ ]:
d = embeddings.shape[1] # 384
index = faiss.IndexFlatL2(d)
index.add(embeddings)

## Downloading Gemma 2B instruction tuned

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers

model_id = "google/gemma-2b-it"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda",
    torch_dtype=dtype,
)

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## Functions to generate the prompts (RAG and no RAG)

In [ ]:
def create_simple_prompt(user_input):
  chat = [
      { "role": "user", "content": user_input },
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  return prompt


def create_rag_prompt(user_input, embedding_model, docs, db, k=5):

  embedding = embedding_model.encode([user_input])
  _, I = db.search(embedding, k)

  context = "Relevant documents:\n"
  for i in I[0]:
    context += f"Doc {i+1}: {docs[i]}\n"

  chat = [
      { "role": "user", "content": f"Use the documents to answer the user.\n{context}\n{user_input}"},
  ]
  prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
  return prompt

## Running the pipeline

### No RAG

In [ ]:
query = "who are the authors of pymatting?"

prompt = create_simple_prompt(query)
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
print(tokenizer.decode(outputs[0]).split("<start_of_turn>model")[1])


The authors of Pymatting are:

* **David MacDiarmid**
* **Mark P. Allen**
* **David R. Reichman**
* **Michael F. Crompton**
* **Daniel R. Reichman**<eos>


In [ ]:
print(prompt)

<bos><start_of_turn>user
who are the authors of pymatting?<end_of_turn>
<start_of_turn>model



### Using RAG

In [ ]:
query = "who are the authors of pymatting?"

prompt = create_rag_prompt(query, embedding_model, all_chunks, index, k=10)
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)
print(tokenizer.decode(outputs[0]).split("<start_of_turn>model")[1])


The authors of PyMatting are Thomas Germer, Tobias Uelwer, Stefan Conrad, and Stefan Harmeling.<eos>


In [ ]:
print(prompt)

<bos><start_of_turn>user
Use the documents to answer the user.
Relevant documents:
Doc 155: readmes/pymatting.mdich are 100% foreground.\nAll other values indicate unknown pixels which will be estimated by the algorithm.\nTesting\nRun the tests from the main directory:\n python3 tests/download_images.py\n pip3 install -r requirements_tests.txt\n pytest\n\nCurrently 89% of the code is covered by tests.\nUpgrade\npip3 install --upgrade pymatting\npython3 -c \"import pymatting\"\nBug Reports, Questions and Pull-Requests\nPlease, see our community guidelines.\nAuthors\n\nThomas Germer\nTobias Uelwer\nStefan Conrad\nStefan Harmeling\n\nSee also the list of contributors who participated in this project.\nProjects using PyMatting\n\nRembg - an excellent tool for removing image backgrounds.\nPaddleSeg - a library for a wide range of image segmentation tasks.\nchaiNNer - a node-based image processing GUI.\nL
Doc 147: readmes/pymatting.mdse,"configFilePath":null,"networkDependabotPath":"/pymatti